In [37]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_pydot import write_dot
import networkx.algorithms.isomorphism as iso


def raw_possibilities_with_one_more_loop(G):
    n = G.number_of_nodes()
    G.add_nodes_from([
        (n, {"color":"blue"}),
        (n+1, {"color":"blue"})
    ])
    solutions = []
    for edge in G.edges:
        G_new = nx.MultiGraph()
        G_new.add_nodes_from(G.nodes(data=True))
        G_new.add_edges_from(G.edges)
        G_new.add_edges_from([
            (edge[0], n),
            (n, n+1),
            (n, n+1),
            (n+1, edge[1])
        ])
        G_new.remove_edge(edge[0], edge[1])
        solutions.append(G_new)
    return solutions
    
def eliminate_isomorphic_solutions(solutions):
    """
    creates a list with all graphs from solutions that are not isomorphic via is_feynman_isomorphic
    """
    new_solutions = solutions[:1]
    for sol in solutions[1:]:
        append = True
        for ref_sol in new_solutions:
            if is_feynman_isomorpic(sol, ref_sol):
                append = False
        if append:
            new_solutions.append(sol)
    return new_solutions
    
def is_feynman_isomorpic(G1, G2):
    nm = iso.categorical_node_match("color", "blue")
    return nx.is_isomorphic(G1, G2, node_match=nm)

def add_loop(graphs_before):
    solutions = []
    for graph in graphs_before:
        for new_graph in raw_possibilities_with_one_more_loop(graph):
            solutions.append(new_graph)
    return eliminate_isomorphic_solutions(solutions)


In [40]:
G0 = nx.MultiGraph()
G0.add_nodes_from([
    (0, {"color":"red"}),
    (1, {"color":"green"})
])
G0.add_edge(0,1)


solutions = raw_possibilities_with_one_more_loop(G0)
G1 = solutions[0]
solutions = add_loop(solutions)
solutions = add_loop(solutions)
for i, sol in enumerate(solutions):
    print(sol.edges)
    write_dot(sol, f'visualization/graph{i}.dot')

[(0, 6, 0), (1, 3, 0), (2, 3, 0), (2, 3, 1), (2, 5, 0), (4, 5, 0), (4, 5, 1), (4, 7, 0), (6, 7, 0), (6, 7, 1)]
[(0, 4, 0), (1, 3, 0), (2, 3, 0), (2, 5, 0), (2, 6, 0), (3, 7, 0), (4, 5, 0), (4, 5, 1), (6, 7, 0), (6, 7, 1)]
[(0, 4, 0), (1, 3, 0), (2, 3, 0), (2, 3, 1), (2, 5, 0), (4, 5, 0), (4, 6, 0), (5, 7, 0), (6, 7, 0), (6, 7, 1)]
[(0, 2, 0), (1, 3, 0), (2, 4, 0), (2, 6, 0), (3, 5, 0), (3, 7, 0), (4, 5, 0), (4, 5, 1), (6, 7, 0), (6, 7, 1)]
[(0, 2, 0), (1, 3, 0), (2, 3, 0), (2, 6, 0), (3, 5, 0), (4, 5, 0), (4, 5, 1), (4, 7, 0), (6, 7, 0), (6, 7, 1)]
[(0, 2, 0), (1, 3, 0), (2, 3, 0), (2, 4, 0), (3, 5, 0), (4, 5, 0), (4, 6, 0), (5, 7, 0), (6, 7, 0), (6, 7, 1)]


In [39]:
import os
for filename in os.listdir('visualization'):
    if filename[-4:] == ".dot":
        newfilename = filename[:-4] + '.png'
        os.system(f'neato -T png visualization/{filename} > visualization/{newfilename}')

Wie wächst die Anzahl an Graphen mit der Anzahl der Löcher? schneller oder langsamer als exponentiell?
Graph: 
- 2-verbunden
- ungerichtet
- muss nicht planar sein
- 3-regulär (jeder Knoten hat 3 Kanten)
- feste Anzahl an Halbkanten

Inzwischen können wir die Anzahl der einfachen 3-regulären Graphen für n Knoten asymptotisch abschätzen.
Durch Aufschneiden eines Knotens bekommen wir höchstens 6 verschiedene Feynman-Diagramme.
Da die Automorphismengruppe asymptotisch trivial ist, erhalten wir also eine asymptotische obere Schranke für die Anzahl der drei-regulären zusammenhängenden Feynman-Diagramme. Nun müssen wir noch die zwei-Verbundenheit testen bzw. ermitteln, wie sich das auf die asymptotische Verteilung auswirkt.

Aus https://www.sciencedirect.com/science/article/pii/0097316578900596 erhalten wir eine Abschätzung für die Anzahl der k-valenten Graphen.
Es muss nun noch geklärt werden, wie sich diese Anzahl verändert, wenn wir nur die 2-zusammenhängenden Graphen betrachten. 
Aus https://web.williams.edu/Mathematics/sjmiller/public_html/ntprob13/handouts/graphs/Womald_ModelsRandomGraphs.pdf, 
Abschnitt 2.6 folgt, dass die Graphen asymptotisch fast alle mindestens 2-connected sind.

Interessantes paper, was ähnliche Dinge macht wie wir auch: https://arxiv.org/pdf/2305.13506.pdf